In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import xlrd
import pickle
from pandas.tseries.offsets import MonthEnd
import calendar
import country_converter as coco
import os

In [7]:

# Options for loading and saving
LoadDataType = 'Ours' # options: 'Ours','Test','Stanford'
#saveAppend = '_OurData_NScode'
saveAppend = ''




if LoadDataType=='Test':
    # IMF codes for spot rate countries
    country_codes_spot = ['914', '612', '213', '193', '122', '419', '513', '124', '66666',
                      '218', '616', '223', '516', '918', '618', '156', '228', '924', '233', 
                      '662', '960', '423', '935', '128', '163', '248', '469', '939', '172', 
                      '819', '132', '648', '134', '652', '174', '656', '532', '944', '176', 
                      '534', '536', '178', '436', '136', '158', '439', '916', '664', '443', 
                      '941', '446', '946', '137', '676', '548', '181', '682', '273', '686', 
                      '688', '138', '196', '694', '142', '449', '564', '853', '288', '293', 
                      '566', '964', '182', '453', '968', '922', '456', '576', '961', '199', 
                      '542', '184', '524', '144', '146', '528', '578', '744', '186', '466', 
                      '746', '926', '298', '846', '299', '582', '754', '698', '112']
    
    
    # Set current directory as path_rawdata
    path_rawdata = os.path.normpath(os.getcwd()+os.sep+os.pardir)+'/RawData/'

    # Import Reuters forward rates
    # FX_Fwd_temp = pd.read_excel(path_rawdata+'Reuters_D.xlsx', sheet_name='Fwd1', skiprows=1)

    # Import Reuters forward rates (using my tester dataset)
    FX_Fwd_temp = pd.read_excel(path_rawdata+'Reuters_D_fortesting.xlsx', sheet_name='Fwd1', skiprows=1)
                                    
    # Convert date to datetime object (from excel format which counts days since 1900)
    date_column = pd.to_datetime(FX_Fwd_temp['Code'], unit='D', origin='1899-12-31')
    FX_Fwd_temp.insert(0, 'Date', date_column)
    FX_Fwd_temp = FX_Fwd_temp.drop(['Code'],axis=1)

    # # IMF codes for forward rate countries
    country_codes_fwd = ['466', '122', '193', '124', '156', '146', '935', '134', '128', '163', '184',
                     '172', '132', '112', '174', '532', '536', '178', '136', '158', '443', '273',
                     '548', '138', '142', '186', '528', '199', '944', '534', '964', '576', '578',
                     '196', '542', '566', '182', '456']
    # Change column names to Date (for first column)
    column_names_fwd = ['Date'] + country_codes_fwd
    FX_Fwd_temp.columns = column_names_fwd


    # Import Reuters spot rates
    # FX_Spot_temp = pd.read_excel(path_rawdata+'Reuters_D.xlsx', sheet_name='Spot', skiprows=1)

    # Import Reuters spot rates (using my tester dataset)
    FX_Spot_temp = pd.read_excel(path_rawdata+'Reuters_D_fortesting.xlsx', sheet_name='Spot', skiprows=1)

    # Convert date to datetime object (from excel format which counts days since 1900)
    date_column = pd.to_datetime(FX_Spot_temp['Code'], unit='D', origin='1899-12-31')
    FX_Spot_temp.insert(0, 'Date', date_column)
    FX_Spot_temp = FX_Spot_temp.drop(['Code'],axis=1)

    # get rid of special drawing rights column (SPEDRAW(ER))
    FX_Spot_temp.drop(columns=['SPEDRAW(ER)'], inplace=True)

    column_names_spot = ['Date'] + country_codes_spot

    # Change column names to Date (for first column) and IMF country codes (for all other columns)
    FX_Spot_temp.columns = column_names_spot
    
    path_codeNames =path_rawdata
elif LoadDataType=='Ours':
     # Set current directory as path_rawdata
    path_rawdata = os.path.normpath(os.getcwd()+os.sep+os.pardir)+"/RawData/"
    
    # Import Barclays forward rates (using actual dataset)
    #FX_Fwd_temp = pd.read_excel(path_rawdata+'Datastream & Barclays - Spots & 1M Forwards.xlsx', sheet_name='RFV FX Fwd', skiprows=0)
    FX_Fwd_temp = pd.read_excel(path_rawdata+'Datastream & Barclays - Spots & 1M Forwards.xlsx', sheet_name='RFV FX Fwd Outright', skiprows=0)
    FX_Fwd_temp['Date'] =pd.to_datetime(FX_Fwd_temp['Date'])
    
    # Convert column names to country names (Nick comment: I don't like the hard coded method of the fake data)
    colDetails =  pd.read_excel(path_rawdata+'Exchange Rates - Datastream Codes.xlsx', sheet_name='RFV FX Fwd Outright Codes', skiprows=0)
    colDetails = colDetails[['Symbol','Country']]
    colDetails = colDetails[colDetails['Symbol'].isin(FX_Fwd_temp.columns)]
    
    # fix couple of outlier labels
    colDetails.loc[colDetails['Country']=='Euro','Country']='Euro Area'
    colDetails.loc[colDetails['Country']=='Euro (inverse)','Country']='Euro Area (INVERSE)'
    colDetails.loc[colDetails['Country']=='Hong Kong','Country']='CHINA HONG KONG'
    colDetails.loc[colDetails['Country']=='Estonia','Country']='ESTONIAN'
    colDetails.loc[colDetails['Country']=='Jordan','Country']='JORDANIA'
    colDetails.loc[colDetails['Country']=='Serbia','Country']='SERBIA AND MONTENEGRO'
   
    # make upper case
    colDetails.loc[:,'Country']=colDetails['Country'].str.upper()

    # drop duplicate Japan, Switzerland and Russia series 
    FX_Fwd_temp = FX_Fwd_temp.drop(columns=['TDJP21M','TDCH21M','TDRU21M','TDRU11M'])

    # make all upper case
    FX_Fwd_temp =FX_Fwd_temp.rename(columns = colDetails.set_index('Symbol')['Country'])
    
    for nam in FX_Fwd_temp.columns:
        if '(INVERSE)' in nam:
            # remove inverse from name
            newNam = nam.replace(' (INVERSE)','')
            FX_Fwd_temp=FX_Fwd_temp.rename(columns={nam:newNam})
            
            # invert series
            FX_Fwd_temp.loc[:,newNam]=1.0/FX_Fwd_temp.loc[:,newNam]
            
            
    # now replace country with IMF code
    IMF_codes = pd.read_excel(path_rawdata+'IMF_codes.xlsx',header=None,names=['Country', 'IMF Code'])
    FX_Fwd_temp = FX_Fwd_temp.rename(columns = IMF_codes.set_index('Country')['IMF Code'].astype('str'))
   
    
    # Now repeat the same with the spot data!
    FX_Spot_temp = pd.read_excel(path_rawdata+'Datastream & Barclays - Spots & 1M Forwards.xlsx', sheet_name='RFV FX Spot', skiprows=0)

    
    # Convert column names to country names (Nick comment: I don't like the hard coded method of the fake data)
    colDetails =  pd.read_excel(path_rawdata+'Exchange Rates - Datastream Codes.xlsx', sheet_name='RFV Spot Codes', skiprows=0)
    colDetails = colDetails[['Symbol','Country']]
    colDetails = colDetails[colDetails['Symbol'].isin(FX_Spot_temp.columns)]
    
    # fix couple of outlier labels
    colDetails.loc[colDetails['Country']=='Euro','Country']='Euro Area'
    colDetails.loc[colDetails['Country']=='Euro (inverse)','Country']='Euro Area (INVERSE)'
    colDetails.loc[colDetails['Country']=='Hong Kong','Country']='CHINA HONG KONG'
    colDetails.loc[colDetails['Country']=='Estonia','Country']='ESTONIAN'
    colDetails.loc[colDetails['Country']=='Jordan','Country']='JORDANIA'
    colDetails.loc[colDetails['Country']=='Serbia','Country']='SERBIA AND MONTENEGRO'
    colDetails.loc[colDetails['Country']=='Venezuela Soberano','Country']='VENEZUELA'
    colDetails.loc[colDetails['Country']=='Venezuela Fuerte','Country']='VENEZUELA'
    colDetails.loc[colDetails['Country']=='Cape Verde','Country']='CABO VERDE'
    colDetails.loc[colDetails['Country']=='Bosnia','Country']='BOSNIA AND HERZEGOWINA'
    colDetails.loc[colDetails['Country']=='Central Africa','Country']='CENTRAL AFRICAN REPUBLIC'
    colDetails.loc[colDetails['Country']=='Serbia','Country']='SERBIA AND MONTENEGRO'
#    colDetails.loc[colDetails['Country']=='Mauritania (Old)','Country']='MAURITANIA'
    colDetails.loc[colDetails['Country']=='Norwegian','Country']='NORWAY'
    colDetails.loc[colDetails['Country']=='Jordan','Country']='JORDANIA'
    colDetails.loc[colDetails['Country']=='Jordan','Country']='JORDANIA'
    colDetails.loc[colDetails['Country']=='Jordan','Country']='JORDANIA'

    
    
    # remove duplicate NZ, Australia  and Mexico columns
    FX_Spot_temp = FX_Spot_temp.drop(columns=['TDNZDSP','MXPSUF.','TDAUDSP'])
    
    
    # make all upper case
    colDetails.loc[:,'Country']=colDetails['Country'].str.upper()
    
    FX_Spot_temp =FX_Spot_temp.rename(columns = colDetails.set_index('Symbol')['Country'])
    FX_Spot_temp=FX_Spot_temp.rename(columns={'Code':'Date'})
    FX_Spot_temp['Date'] =pd.to_datetime(FX_Spot_temp['Date'])
    
    FX_Spot_temp = FX_Spot_temp.drop(columns=['SIERRA LEONE (OLD)','MAURITANIA (OLD)','CUBA','NGN PARALLEL','DELETE','NORTH KOREA','EAST CARIBBEAN','WEST AFRICA','VENEZUELA','FRENCH PACIFIC','CAYMAN ISLANDS'])
    
    for nam in FX_Spot_temp.columns:
        if '(INVERSE)' in nam:
            # remove inverse from name
            newNam = nam.replace(' (INVERSE)','')
            FX_Spot_temp=FX_Spot_temp.rename(columns={nam:newNam})
            
            # invert series
            FX_Spot_temp.loc[:,newNam]=1.0/FX_Spot_temp.loc[:,newNam]
    

     # now replace country with IMF code
    FX_Spot_temp = FX_Spot_temp.rename(columns = IMF_codes.set_index('Country')['IMF Code'].astype('str'))
    
    path_codeNames =path_rawdata
    # IMF codes for spot rate countries
    country_codes_spot = FX_Spot_temp.columns[1:]
    
elif LoadDataType=='Stanford':
         # Set current directory as path_rawdata
    path_rawdata = os.path.normpath(os.getcwd()+os.sep+os.pardir)+'/RawData/Stanford Data/ToUpdate/'
    path_codeNames = os.path.normpath(os.getcwd()+os.sep+os.pardir+os.sep+os.pardir)+"/Nick and Jamie's Barclays and Reuters/Raw and processed data/toUse/"
    
    # Import Barclays forward rates (using actual dataset)
    #FX_Fwd_temp = pd.read_excel(path_rawdata+'Datastream & Barclays - Spots & 1M Forwards.xlsx', sheet_name='RFV FX Fwd', skiprows=0)
    FX_Fwd_temp_late = pd.read_excel(path_rawdata+'DataRequests_Reuters_FR_D_since_31_12_2008.xlsm', sheet_name='Sheet1', skiprows=1)
    FX_Fwd_temp_early = pd.read_excel(path_rawdata+'DataRequests_Reuters_FR_D_until_31_12_2008.xlsm', sheet_name='Sheet1', skiprows=1)
    
    # Convert date to datetime object (from excel format which counts days since 1900)
    date_column = pd.to_datetime(FX_Fwd_temp_early['Code'], unit='D', origin='1899-12-31')
    FX_Fwd_temp_early.insert(0, 'Date', date_column)
    FX_Fwd_temp_early = FX_Fwd_temp_early.drop(['Code'],axis=1)
    
    # Convert date to datetime object (from excel format which counts days since 1900)
    date_column = pd.to_datetime(FX_Fwd_temp_late['Code'], unit='D', origin='1899-12-31')
    FX_Fwd_temp_late.insert(0, 'Date', date_column)
    FX_Fwd_temp_late = FX_Fwd_temp_late.drop(['Code'],axis=1)
    
    FX_Fwd_temp = pd.concat([FX_Fwd_temp_early.set_index('Date'),FX_Fwd_temp_late.set_index('Date')]).reset_index()
    
    # Convert column names to country names (Nick comment: I don't like the hard coded method of the fake data)
    colDetails =  pd.read_excel(path_codeNames+'Exchange Rates - Datastream Codes.xlsx', sheet_name='WM RFV Closing Fwd Codes', skiprows=0)
    colDetails = colDetails[['Symbol','Country']]
    colDetails = colDetails[colDetails['Symbol'].isin(FX_Fwd_temp.columns)]
    
    # fix couple of outlier labels
    colDetails.loc[colDetails['Country']=='Euro','Country']='Euro Area'
    colDetails.loc[colDetails['Country']=='Euro (inverse)','Country']='Euro Area (INVERSE)'
    colDetails.loc[colDetails['Country']=='Hong Kong','Country']='CHINA HONG KONG'
    colDetails.loc[colDetails['Country']=='Estonia','Country']='ESTONIAN'
    colDetails.loc[colDetails['Country']=='Jordan','Country']='JORDANIA'
    colDetails.loc[colDetails['Country']=='Serbia','Country']='SERBIA AND MONTENEGRO'
    colDetails.loc[colDetails['Country']=='Venezuela Soberano','Country']='VENEZUELA'
    colDetails.loc[colDetails['Country']=='Bosnia','Country']='BOSNIA AND HERZEGOWINA'
    
    
    # make upper case
    colDetails.loc[:,'Country']=colDetails['Country'].str.upper()

    # drop duplicate UK series 
    FX_Fwd_temp = FX_Fwd_temp.drop(columns=['USGBP1F'])

    # make all upper case
    FX_Fwd_temp =FX_Fwd_temp.rename(columns = colDetails.set_index('Symbol')['Country'])

    
    for nam in FX_Fwd_temp.columns:
        if '(INVERSE)' in nam:
            # remove inverse from name
            newNam = nam.replace(' (INVERSE)','')
            FX_Fwd_temp=FX_Fwd_temp.rename(columns={nam:newNam})
            
            # invert series
            FX_Fwd_temp.loc[:,newNam]=1.0/FX_Fwd_temp.loc[:,newNam]
            
      
    # now replace country with IMF code
    IMF_codes = pd.read_excel(path_codeNames+'IMF_codes.xlsx',header=None,names=['Country', 'IMF Code'])
    FX_Fwd_temp = FX_Fwd_temp.rename(columns = IMF_codes.set_index('Country')['IMF Code'].astype('str'))
    
    # Now repeat the same with the spot data!
    #FX_Spot_temp = pd.read_excel(path_rawdata+'Datastream & Barclays - Spots & 1M Forwards.xlsx', sheet_name='RFV FX Fwd', skiprows=0)
    FX_Spot_temp_late = pd.read_excel(path_rawdata+'DataRequests_Reuters_SP_D_since_12_31_2008.xlsm', sheet_name='Sheet1', skiprows=1)
    FX_Spot_temp_early = pd.read_excel(path_rawdata+'DataRequests_Reuters_SP_D_until_12_31_2008.xlsm', sheet_name='Sheet1', skiprows=1)

    # Convert date to datetime object (from excel format which counts days since 1900)
    date_column = pd.to_datetime(FX_Spot_temp_early['Code'], unit='D', origin='1899-12-31')
    FX_Spot_temp_early.insert(0, 'Date', date_column)
    FX_Spot_temp_early = FX_Spot_temp_early.drop(['Code'],axis=1)
    
    # Convert date to datetime object (from excel format which counts days since 1900)
    date_column = pd.to_datetime(FX_Spot_temp_late['Code'], unit='D', origin='1899-12-31')
    FX_Spot_temp_late.insert(0, 'Date', date_column)
    FX_Spot_temp_late = FX_Spot_temp_late.drop(['Code'],axis=1)
    
    FX_Spot_temp = pd.concat([FX_Spot_temp_early.set_index('Date'),FX_Spot_temp_late.set_index('Date')]).reset_index()
   
    # remove (ER) from column names
    nams = FX_Spot_temp.columns
    for nam in nams:
        newNam = nam.replace('(ER)','')
        
        FX_Spot_temp=FX_Spot_temp.rename(columns={nam:newNam})
        if 'Unnamed' in nam:
            FX_Spot_temp=FX_Spot_temp.drop(columns=nam)

  
    
    # Convert column names to country names (Nick comment: I don't like the hard coded method of the fake data)
    colDetails =  pd.read_excel(path_codeNames+'Exchange Rates - Datastream Codes.xlsx', sheet_name='WM RFV FX Closing Spot Codes', skiprows=0)
    
    # duplicates in our data are the only series here in the Stanford Data
    colDetails.loc[colDetails['Country']=='DELETE','Country']=colDetails.loc[colDetails['Country']=='DELETE','To Country']+' (inverse)'
    
    
    colDetails = colDetails[['Symbol','Country']]
    colDetails = colDetails[colDetails['Symbol'].isin(FX_Spot_temp.columns)]
    
    # fix couple of outlier labels
    colDetails.loc[colDetails['Country']=='Euro','Country']='Euro Area'
    colDetails.loc[colDetails['Country']=='Central Africa','Country']='CENTRAL AFRICAN REPUBLIC'
    colDetails.loc[colDetails['Country']=='Papua New Guinea','Country']='CENTRAL AFRICAN REPUBLIC'
    colDetails.loc[colDetails['Country']=='Euro (inverse)','Country']='Euro Area (INVERSE)'
    colDetails.loc[colDetails['Country']=='Hong Kong','Country']='CHINA HONG KONG'
    colDetails.loc[colDetails['Country']=='Estonia','Country']='ESTONIAN'
    colDetails.loc[colDetails['Country']=='Jordan','Country']='JORDANIA'
    colDetails.loc[colDetails['Country']=='Serbia','Country']='SERBIA AND MONTENEGRO'

    
    # remove duplicate NZ, Australia  and Mexico columns
    #FX_Spot_temp = FX_Spot_temp.drop(columns=['TDNZDSP','MXPSUF.','TDAUDSP'])
    
    
    # make all upper case
    colDetails.loc[:,'Country']=colDetails['Country'].str.upper()
    
    FX_Spot_temp =FX_Spot_temp.rename(columns = colDetails.set_index('Symbol')['Country'])
    FX_Spot_temp=FX_Spot_temp.rename(columns={'Code':'Date'})
    FX_Spot_temp['Date'] =pd.to_datetime(FX_Spot_temp['Date'])
    
    for nam in FX_Spot_temp.columns:
        if '(INVERSE)' in nam:
            # remove inverse from name
            newNam = nam.replace(' (INVERSE)','')
            FX_Spot_temp=FX_Spot_temp.rename(columns={nam:newNam})
            
            # invert series
            FX_Spot_temp.loc[:,newNam]=1.0/FX_Spot_temp.loc[:,newNam]
    
          
     # now replace country with IMF code
    FX_Spot_temp = FX_Spot_temp.rename(columns = IMF_codes.set_index('Country')['IMF Code'].astype('str'))
    
    FX_Spot_temp = FX_Spot_temp.drop(columns=['SPECIAL DRAWING RIGHT'])
    
    country_codes_spot = FX_Spot_temp.columns[1:]
else:
    print('ERROR!!! LoadDataType not entered properly!!!')
    error()
    
FX_Spot = FX_Spot_temp.copy()
FX_Fwd = FX_Fwd_temp.copy()

In [ ]:
IMF_codes[IMF_codes['Country']=='EURO AREA']

In [13]:
# Create dictionaries out of excel file with IMF codes
IMF_codes = pd.read_excel(path_codeNames+'IMF_codes.xlsx',header=None,names=['Country', 'IMF Code'])
IMF_dict = IMF_codes.set_index('Country').to_dict()['IMF Code']
IMF_dict_inv = IMF_codes.set_index('IMF Code').to_dict()['Country']

# using spot code countries only:
country_codes_num = [eval(i) for i in country_codes_spot]
# Create list of countries in Barclays dataset (to be saved)
List_names = [IMF_dict_inv[key] for key in country_codes_num]


In [14]:
# Adjust exchange rate units (FX Forward and spot data are in units of foreign currency 
# per USD except for Australian Dollar (193), Botswana (616), Euro (163), UK (112), 
# New Zealand Dollar (196), Irish Punt (178)).
# adjust so all forward and spot data are in foreign currency per USD
if LoadDataType=='Test':
    countries_to_adjust = ['AUSTRALIA', 'BOTSWANA', 'EURO AREA', 'UNITED KINGDOM', 'NEW ZEALAND', 'IRELAND']

    for country in countries_to_adjust:
        col_country = IMF_dict[country]
        if str(col_country) in FX_Spot.columns:
            # print(str(col_country) + ' is in FX_Spot')
            FX_Spot[str(col_country)] = 1.0/FX_Spot[str(col_country)]
        if str(col_country) in FX_Fwd.columns:
            # print(str(col_country) + ' is in FX_Fwd')
            FX_Fwd[str(col_country)] = 1.0/FX_Fwd[str(col_country)]
        

In [15]:
# function to remove outliers (forward rates only for select countries, below)
def remove_outliers(startdate, enddate, country):
    col_country = IMF_dict[country]
    FX_Fwd.loc[(FX_Fwd.Date>=pd.to_datetime(startdate,infer_datetime_format=True)) & (FX_Fwd.Date<=pd.to_datetime(enddate,infer_datetime_format=True)),[str(col_country)]] = np.nan

In [16]:
# ----------------------------------------------------------------------
# --------------------------- Corrections ------------------------------
# ----------------------------------------------------------------------
# For all the corrections below, need to change dates once use real data
# ----------------------------------------------------------------------

# deleting euro countries
# after 12/31/1998, keep only euro data, write NaN for the euro countries
# excluding greece, delete for spot rates only
# after 01/31/2001, write NaN for Greece (174), delete for spot rates only
eurocountries = ['BELGIUM', 'GERMANY', 'GREECE', 'SPAIN', 'FRANCE', 'IRELAND', 
                  'ITALY', 'LUXEMBOURG', 'NETHERLANDS', 'AUSTRIA', 
                  'PORTUGAL', 'FINLAND']
# endsomeeuro_spot = '1/21/1997' # for testing only
# endgreece_spot = '1/24/1997' # for testing only
endsomeeuro_spot = '12/31/1998'
endgreece_spot = '01/31/2001'
for country in eurocountries:
    col_country = IMF_dict[country]
    if country != 'GREECE':
        FX_Spot.loc[FX_Spot.Date>=pd.to_datetime(endsomeeuro_spot),str(col_country)] = np.nan
    elif country == 'GREECE':
        FX_Spot.loc[FX_Spot.Date>=pd.to_datetime(endgreece_spot),str(col_country)] = np.nan

        
# keep only countries for which we have both forward and spot exchange rates
spotcolumns = list(FX_Spot.columns)
fwdcolumns = list(FX_Fwd.columns)
columnsinboth = list(set(spotcolumns) & set(fwdcolumns))
columnsinboth.insert(0, columnsinboth.pop(columnsinboth.index('Date')))
FX_Spot = FX_Spot[columnsinboth]
FX_Fwd = FX_Fwd[columnsinboth]


# remove outliers in forward rates

# indonesia: remove from 29-Dec-2000 to 31-May-2007
# remove_outliers('1/19/1997', '1/25/1997', 'INDONESIA') #for testing only
remove_outliers('29/12/2000', '31/05/2007', 'INDONESIA')

# south africa: remove from 31-Jul-1985 to 30-Aug-1985
# remove_outliers('1/01/1997', '1/10/1997', 'SOUTH AFRICA') #for testing only
remove_outliers('31/07/1985', '30/08/1985', 'SOUTH AFRICA')

# turkey: remove from 31-Oct-2000 to 30-Nov-2001
# remove_outliers('1/01/1997', '1/10/1997', 'TURKEY') #for testing only
remove_outliers('31/10/2000', '30/11/2001', 'TURKEY')

# malaysia: remove from 31-Aug-1998 to 30-Jun-2005
# remove_outliers('1/05/1997', '1/15/1997', 'MALAYSIA') #for testing only
remove_outliers('31/08/1998', '30/06/2005', 'MALAYSIA')

# UAE: remove from 30-Jun-2006 to 30-Nov-2006
# remove_outliers('1/20/1997', '1/30/1997', 'UNITED ARAB EMIRATES') #for testing only
remove_outliers('30/06/2006', '30/11/2006', 'UNITED ARAB EMIRATES')

In [17]:
# the reuters data is not extended to end of month
Reuters_FX_Fwd_D = FX_Fwd.copy()
Reuters_FX_Spot_D = FX_Spot.copy()

In [18]:
# Save using pickle
# Set current directory as newpath
path_cleandata = os.path.normpath(os.getcwd()+os.sep+os.pardir)+'/CleanData/'

# saving my cleaned tester data
Reuters_FX_Fwd_D.to_pickle(path_cleandata+'Reuters_FX_Fwd_D'+saveAppend+'.pkl')
Reuters_FX_Spot_D.to_pickle(path_cleandata+'Reuters_FX_Spot_D'+saveAppend+'.pkl')

# with open(path_cleandata+'Reuters_Countries.pkl', 'wb') as f:
#     pickle.dump(List_names, f)